In [ ]:
print("Running")

In [ ]:
import gym
import glob
import torch
import matplotlib.pyplot as plt

In [ ]:
deg2rad = lambda x: x * 3.1415926536 / 180.
rad2deg = lambda x: x * 180. / 3.1415926536

In [ ]:
env = gym.make('gym_swirl:swirl-v1')
env.seed(42)

In [ ]:
attempt = 11
rl_type = "acdd"#"mfmaac"#"ac"

In [ ]:
set([p.split("mfmaac").pop().split("_train")[0] for p in glob.glob(f"runs/{rl_type}*")])

In [ ]:
paths = sorted(glob.glob(f"runs/{rl_type}{attempt:03d}_*"), key=lambda x: int(x.split("epoch").pop().split("_2021")[0]))
len(paths)

In [ ]:
hist = torch.zeros(180)
hist_amount = torch.zeros(180)
for path in paths[::1]:
    env.load(path)
    for state in env.states[1:]:
        indexes = rad2deg(state.Deltas).type(torch.int64)
        if len(indexes) == 1:
            index = indexes.item()
            if 0 <= indexes < 180:
                hist_amount[index] += 1
                hist[index] += state.O_R.mean()
        else:
            valid = (indexes >= 0) & (indexes < 180) 
            indexes = indexes[valid]
            
            ors = state.O_R[valid]
            
            hist.index_add_(0, indexes, ors)
            
            hist_amount.index_add_(0, indexes, torch.ones_like(indexes).type(torch.float32))

In [ ]:
plt.rcParams.update({'font.size': 22})
fig = plt.figure(figsize=(20,10))
res = (hist/torch.maximum(hist_amount, torch.tensor([1.])))
plt.plot(res.abs(), c="black")
plt.xlabel("$\Delta$ °")
plt.ylabel("Avg. <$\|O_R\|$>")
plt.title(f"Maximum: $\Delta$ = {res.argmax()}°")
#fig.savefig('avg_or_distr_ac.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
(hist/torch.maximum(hist_amount, torch.tensor([1.]))).abs().argmax()